<a href="https://colab.research.google.com/github/christopherdiamana/nlp/blob/main/Catch-up_2/catch_up2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Natural Language Processing Catch-up 2

## Closing on the sentiment classifier 

In [1]:
import torch
torch.cuda.is_available()

True

### Library and dataset

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 3.9 MB/s 
     |████████████████████████████████| 140 kB 73.1 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 1.1 MB 70.8 MB/s 
     |████████████████████████████████| 596 kB 91.5 MB/s 
     |████████████████████████████████| 127 kB 26.1 MB/s 
     |████████████████████████████████| 271 kB 79.5 MB/s 
     |████████████████████████████████| 144 kB 73.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [3]:
from datasets import get_dataset_split_names

In [4]:
get_dataset_split_names("imdb")

['train', 'test', 'unsupervised']

In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

#### Split the training set into a training and validation set

In [8]:
dataset_clean = dataset["train"].train_test_split(train_size=0.8, stratify_by_column="label")

# Rename the default "test" split to "validation"
dataset_clean["validation"] = dataset_clean.pop("test")

# Add the "test" set to our `DatasetDict`
dataset_clean["test"] = dataset["test"]

In [9]:
dataset_clean

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [10]:
dataset_clean['train'].features

{'label': ClassLabel(num_classes=2, names=['neg', 'pos'], id=None),
 'text': Value(dtype='string', id=None)}

#### Let's check if the proportion of each class must be the same in the training and validation set

In [11]:
from collections import Counter

In [12]:
Counter(dataset_clean['train']['label'])

Counter({0: 10000, 1: 10000})

In [13]:
Counter(dataset_clean['validation']['label'])

Counter({0: 2500, 1: 2500})

### Fine-tuning a model

#### 1. Fine-tune the `distilbert-base-uncased` model on the training data

In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 63.1 MB/s 


In [15]:
from transformers import AutoTokenizer

In [16]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

With the option `batched=True` I will preprocessed faster 

In [18]:
tokenized_datasets = dataset_clean.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7f5afb476b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [20]:
tokenized_datasets = tokenized_datasets.shuffle(seed=42)

##### Preprocessing

In [21]:
from transformers import DataCollatorWithPadding

In [22]:
data_collator = DataCollatorWithPadding(tokenizer)

##### Training

First step, I will define a `TrainingArguments` class that will contain all the hyperparameters the `Trainer` will use for training and evaluation.

In [23]:
from transformers import TrainingArguments

The only two arguments that I will provide are a directory where the trained model will be saved, as well as the checkpoints along the way and the total number of training epochs to perform because by default num_train_epoch (float) is 3.0 and in my case I would like 1. For all the rest, I will leave the defaults, which should work pretty well for a basic fine-tuning.

In [24]:
directory_name = "basic-finetuning-distilbert"
 
training_args = TrainingArguments(
   output_dir=directory_name,
   num_train_epochs=1,
)

Second step, I will define the model. As in the previous chapter, we will use the `AutoModelForSequenceClassification` class, with two labels:

In [25]:
from transformers import AutoModelForSequenceClassification

In [26]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [27]:
from transformers import Trainer

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [29]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Step,Training Loss
500,0.426900
1000,0.322600
1500,0.298600
2000,0.261400
2500,0.257600


Saving model checkpoint to basic-finetuning-distilbert/checkpoint-500
Configuration saved in basic-finetuning-distilbert/checkpoint-500/config.json
Model weights saved in basic-finetuning-distilbert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in basic-finetuning-distilbert/checkpoint-500/tokenizer_config.json
Special tokens file saved in basic-finetuning-distilbert/checkpoint-500/special_tokens_map.json
Saving model checkpoint to basic-finetuning-distilbert/checkpoint-1000
Configuration saved in basic-finetuning-distilbert/checkpoint-1000/config.json
Model weights saved in basic-finetuning-distilbert/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in basic-finetuning-distilbert/checkpoint-1000/tokenizer_config.json
Special tokens file saved in basic-finetuning-distilbert/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to basic-finetuning-distilbert/checkpoint-1500
Configuration saved in basic-finetuning-distilbert/checkpoint-1500/config.js

TrainOutput(global_step=2500, training_loss=0.31341150512695315, metrics={'train_runtime': 922.0784, 'train_samples_per_second': 21.69, 'train_steps_per_second': 2.711, 'total_flos': 2498142646963488.0, 'train_loss': 0.31341150512695315, 'epoch': 1.0})

#### For what follow, I will use a [fine-tuned](https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb) version of distilbert-base-uncased on an imdb dataset where an evaluation of 5000 samples was created by splitting the training set.

In [30]:
tokenizer_mvonwyl = AutoTokenizer.from_pretrained("mvonwyl/distilbert-base-uncased-imdb")

model_mvonwyl = AutoModelForSequenceClassification.from_pretrained("mvonwyl/distilbert-base-uncased-imdb")

https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpshwetupz


Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c312c0f6745a03b88553cfb3e7603c49b7d3367514a3c63fed0846336510b332.c3f9f3d0063599ad5a8014363c5b46996af3c3efe30fb83a0d33a55b59bc6a23
creating metadata file for /root/.cache/huggingface/transformers/c312c0f6745a03b88553cfb3e7603c49b7d3367514a3c63fed0846336510b332.c3f9f3d0063599ad5a8014363c5b46996af3c3efe30fb83a0d33a55b59bc6a23
https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpv0xo6j0j


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/4281206e7a7e0a9edc5227f33541196292145147630ce901689ac74b449256ed.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/4281206e7a7e0a9edc5227f33541196292145147630ce901689ac74b449256ed.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgf8bpom5


...

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/bda42573df3424e9017803e91fd725748cb1d4b27de381bd7bbdb172eb8666d1.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for /root/.cache/huggingface/transformers/bda42573df3424e9017803e91fd725748cb1d4b27de381bd7bbdb172eb8666d1.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/4281206e7a7e0a9edc5227f33541196292145147630ce901689ac74b449256ed.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/c57072a97ab75bfd6274f08f88500ca960cfe6ca4c9435e9f185599b04e2435d.848c414913cfee271695b876

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/155a1fb3ac6bb82f94f0363f0495d57da88c36b77b545cd468d5381686931fb1.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
creating metadata file for /root/.cache/huggingface/transformers/155a1fb3ac6bb82f94f0363f0495d57da88c36b77b545cd468d5381686931fb1.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
loading configuration file https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/155a1fb3ac6bb82f94f0363f0495d57da88c36b77b545cd468d5381686931fb1.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
Model config DistilBertConfig {
  "_name_or_path": "mvonwyl/distilbert-base-uncased-imdb",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

storing https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/90c7fa3d5a0007b25a637af126d11dabdb935d55c4f9f887c59d95f86e825081.4f5038faecc327c154d58f2b404e38b27e69abca5d2c5e2f39ecf462efeb54b1
creating metadata file for /root/.cache/huggingface/transformers/90c7fa3d5a0007b25a637af126d11dabdb935d55c4f9f887c59d95f86e825081.4f5038faecc327c154d58f2b404e38b27e69abca5d2c5e2f39ecf462efeb54b1
loading weights file https://huggingface.co/mvonwyl/distilbert-base-uncased-imdb/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/90c7fa3d5a0007b25a637af126d11dabdb935d55c4f9f887c59d95f86e825081.4f5038faecc327c154d58f2b404e38b27e69abca5d2c5e2f39ecf462efeb54b1
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at mvonwyl/distilbert-base-uncased-imdb

#### 2. Evaluation

I will evaluate the model in term of accuracy on the test data

In [31]:
trainer_mvonwyl = Trainer(
    model=model_mvonwyl,
    tokenizer=tokenizer_mvonwyl
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


First, I will get some predictions from the model.

In [32]:
predictions = trainer_mvonwyl.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25000
  Batch size = 8


(25000, 2) (25000,)


In [33]:
predictions.metrics

{'test_loss': 0.3972010016441345,
 'test_runtime': 397.7052,
 'test_samples_per_second': 62.861,
 'test_steps_per_second': 7.858}

The output of the `predict()` method contain this three properties: predictions, label_ids, and metrics.

`Predictions` property is a two-dimensional array. Those are the logits for each element of the dataset we passed to `predict()`. 

I will take the index with the maximum value on the second axis to transform the two-dimensional array into one to be able to compare its to the labels:

In [34]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

I can now compare the `preds` to the `labels`

In [35]:
from datasets import load_metric

metric = load_metric("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.92948}

Here, I can see the fine-tuned version of distilbert-base-uncased has an accuracy of 92.94% on the test set.

#### 3. Wrongly classified samples

I will select 2 samples which have been wrongly classified in the test set.

In [36]:
result_imdb_test = preds != predictions.label_ids

In [37]:
wrong_imdb_test = np.where(result_imdb_test == True)
wrong_imdb_test

(array([   21,    30,    53, ..., 24976, 24995, 24998]),)

In [38]:
type(wrong_imdb_test[0])

numpy.ndarray

I will randomly select 2 samples and make sure that will be the same by using `seed`.

In [39]:
np.random.seed(42)
two_wrongly_index = np.random.choice(wrong_imdb_test[0], 2)
two_wrongly_index

array([16121, 20899])

In [40]:
two_wrongly_index[0]

16121

In [41]:
type(tokenized_datasets["test"])

datasets.arrow_dataset.Dataset

In [42]:
def get_information_from_index(index: int, data, predictions: [int]):
  '''
  Get the text, the label and the prdiction label from the index. 
  Diplay those information.
  
      Parameters:
              index (int): A index integer
              texts (Vector): list of the texts 
              predictions (Vector): list of the predictions
  '''
  label = "positive" if data[index]['label'] == 1 else "negative"
  prediction = "positive" if predictions[index] == 1 else "negative"
  text = data[index]['text']

  print("Class:", label)
  print("Prediction:", prediction)
  print("Text:")
  display(text)

In [43]:
get_information_from_index(int(two_wrongly_index[0]), tokenized_datasets["test"], preds)

Class: negative
Prediction: positive
Text:


'I have read the other user comments and I am happy someone has compared it to the original by Kamal called Perumarzhakalam released in 2004.<br /><br />The original had a tight story and no loopholes as described above about the Indian Govt not having proper records, or even bad shoots and bloopers.<br /><br />The story is great and a touchy one and well described by others. But sadly Nagesh taking credit for it as his own story is a sad thing and amounts to nothing other than plagiarism.<br /><br />I guess he has been affected by Bollywood\'s so called "inspired" syndrome.<br /><br />He must at least give credit where it is due.<br /><br />I liked some of his older movies, but now I suspect if any of them were originals after all.<br /><br />Here is a link in IMDb for the original masterpiece. http://www.imdb.com/title/tt0425350/#comment I recommend everyone to see the original, even with subtitles if needed, to know what class direction and class acting is all about.'

In my opinion, the model wrongly classified this negative review because the writer was talking about the original version most of the time. The original version is a movie that he `liked` and `recommand`. In addition, from the beginning, he was `happy`. Regarding the actual version, the writer mentions that it is `nothing other than plagiarism` and `sadly Nagesh taking credit for it as his own story`. So as a reader, we can easily understand that he hates the movie. 

Here the model did not distinguish the original version from the actual version. 

In [44]:
get_information_from_index(int(two_wrongly_index[1]), tokenized_datasets["test"], preds)

Class: positive
Prediction: negative
Text:


"A twist of fate puts a black man at the head of an old-school, white-bred advertising firm. And he intends to make a few changes...<br /><br />One very strange piece of cinema. You'll either love it or hate it. Either way, you've never seen anything like it."

To my mind, the model considered the review as a negative one because there are some words and expressions that sound negetive. For instance, the sentences `One very strange piece of cinema`, `never seen anything like it` and also `hate it` can easily highlight that the review seems negative.




#### 4. What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course?

Advantages: 
*   Time saver: I don't need to implement anything, just do a `predict()` and check the results.


Inconvenient:
*   Training: To use easily a model in production, this supposes that the model was already trained on the dataset used (like the `mvonwyl/distilbert-base-uncased-imdb`). Otherwise, the model has to be trained with a good parameter, number of batches, learning rate, etc... And training can take time depending on the number of epochs.


#### 5. Fine-tune your model

Here, I will fine-tune my model using the accuracy as evaluation instead of the loss (default). I will use the Trainer class. The model should have a better results on the validation set instead my first one.

To calculate the accuracy during the evaluation I have to provide a `compute_metrics()` function to the `Trainer`. Also, I have to create a new `TrainingArguments` with its `evaluation_strategy` set to "epoch".

In [48]:
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [53]:
directory_name = "finetuning-distilbert"
 
finetune_training_args = TrainingArguments(
    output_dir=directory_name,
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    weight_decay=0.01
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [54]:
finetune_trainer = Trainer(
    model=model,
    args=finetune_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [55]:
finetune_trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy
1,0.074000,0.312753,0.919200


Saving model checkpoint to finetuning-distilbert/checkpoint-500
Configuration saved in finetuning-distilbert/checkpoint-500/config.json
Model weights saved in finetuning-distilbert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in finetuning-distilbert/checkpoint-500/tokenizer_config.json
Special tokens file saved in finetuning-distilbert/checkpoint-500/special_tokens_map.json
Saving model checkpoint to finetuning-distilbert/checkpoint-1000
Configuration saved in finetuning-distilbert/checkpoint-1000/config.json
Model weights saved in finetuning-distilbert/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in finetuning-distilbert/checkpoint-1000/tokenizer_config.json
Special tokens file saved in finetuning-distilbert/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenc

TrainOutput(global_step=1250, training_loss=0.09379752044677735, metrics={'train_runtime': 1010.0556, 'train_samples_per_second': 19.801, 'train_steps_per_second': 1.238, 'total_flos': 2624005303914624.0, 'train_loss': 0.09379752044677735, 'epoch': 1.0})